# DisALEXI Collection Example

In [1]:
import ee
from IPython.display import Image

from disalexi import disalexi
from disalexi import landsat
disalexi = reload(disalexi)
landsat = reload(landsat)

ee.Initialize()

ndvi_palette = ['#EFE7E1', '#003300']
# ta_palette = ['#ff0000', '#ffff00', '#00ffff', '#0000ff']
et_palette = [
  'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
  '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']

#### Define the Landsat TOA collection

In [2]:
# scene_id = 'LC08_038038_20150717'
landsat_coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA') \
    .filterDate('2015-06-01', '2015-06-10') \
    .filterMetadata('WRS_PATH', 'equals', 43) \
    .filterMetadata('WRS_ROW', 'equals', 33)

#### Define a study area (for generating thumbnail)

In [8]:
test_image = ee.Image(landsat_coll.first())
image_region = test_image.geometry().bounds(1, 'EPSG:4326').coordinates().getInfo()
image_crs = test_image.select(['B2']).projection().crs().getInfo()
image_transform = test_image.select(['B2']).projection().getInfo()['transform']
coarse_transform = image_transform[:]
coarse_transform[0] = 240
coarse_transform[4] = 240

#### Compute ET collection

In [4]:
def apply_et(landsat_img):
    input_img = ee.Image(landsat.Landsat(landsat_img).prep())
    
    disalexi_obj = disalexi.DisALEXI(
        input_img, 
        iterations=10,
        elevation=ee.Image('USGS/SRTMGL1_003').rename(['elevation']),
        landcover=ee.Image(
            ee.ImageCollection('users/cgmorton/GlobeLand30_2010').mosaic()) \
                .divide(10).floor().multiply(10).rename(['landcover']),
        lc_type='GLOBELAND30')
    
    ta_img = disalexi_obj.compute_ta()
    et_img = disalexi_obj.compute_et(ta_img)
    
    return ee.Image(et_img).copyProperties(
        landsat_img, ['system:time_start', 'system:index'])

et_coll = ee.ImageCollection(landsat_coll.map(apply_et))

In [9]:
# Compute the mean ET for the collection
# Project for display purposes
et_img = ee.Image(et_coll.mean()) \
    .reproject(crs=image_crs, crsTransform=coarse_transform)
Image(url=et_img.getThumbURL({
    'min': 0, 'max': 1.25, 'region': image_region, 'palette': ','.join(et_palette)}))

https://earthengine.googleapis.com/api/thumb?thumbid=43d004693c0665d3b0da1ce494b74cbb&token=bba039fbbcaa9983a34dee721714ed57
